In [1]:
from scipy.stats import norm, multivariate_normal
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import networkx as nx
from tqdm import tqdm
from itertools import repeat
from ipywidgets import *
import pickle
from Classes import *
from Functions import *


def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)   
def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
n = 50
m = 1
s = 1
M  = np.array(n*[np.linspace(0 ,  m, n)])
S = np.array(n*[np.linspace(s,0.001, n)]).T

In [3]:
shifts = [0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,1,1.25,1.5,1.75,2,2.5,3,3.5,4,5,8,10]

In [12]:
shifts = [-0.1,-0.2,-0.3,-0.4,-0.5,-0.6,-0.7]

# Numerical

In [29]:
ResultsSim = load_obj("ResultsSim")
temp = load_obj("ResultsSim1")
ResultsSim.update(temp)
temp = load_obj("ResultsSim2")
ResultsSim.update(temp)

In [8]:
ResultsAna = load_obj("ResultsAna")
#ResultsKarate = load_obj("ResultsKarate")

In [60]:
#save_obj(ResultsSim,"ResultsSim")
#save_obj(ResultsAna,"ResultsAna")

In [9]:
#Runner function that runs simulations for all parameter combinations
def run(Simulator,Results,name,save=False):
    """
    Simulator = Function that runs Simulation for m,s,m0,s0,typ combination
    name = name to store results
    Results = results dict
    save = Bool if results should be saved under name
    """
    for Mshift in tqdm(shifts): 
        try:
            Results[Mshift]
        except:
            Results[Mshift]={}

        for typ in ["out","red"]:
            print(Mshift,typ)
            try:
                Results[Mshift][typ]
            except:
                M0 = M.copy()+Mshift
                S0 = S.copy()
                results =[list(map(Simulator,par, repeat(typ))) for par in tqdm(np.dstack((M,S,M0,S0)), position=0, leave=True)]
                Results[Mshift][typ]=results
                if save:
                    save_obj(Results,name)

In [10]:
def ABM_MFana(par,typ):
    m,s,m0,s0 = par
    num = 500
            
    theta1 = np.linspace(m-3.5*s,m+3.5*s,num)
    theta2 = np.linspace(m0-3.5*s0,m0+3.5*s0,num)
    theta1, theta2 = np.meshgrid(theta1,theta2)
    
    pos = np.empty(theta1.shape + (2,))
    pos[:, :, 0] = theta1; pos[:, :, 1] = theta2
    rv = multivariate_normal([m, m0], [[s**2, 0], [0, s0**2]])        
    P= rv.pdf(pos)
    
    State = np.zeros(theta1.shape)
    X_=[]
    Xt_1 =-1
    Xt_2 =-1
    Xt_3 =-1
    Xt   = double_Integral(theta1, theta2, State*P)
    for i in range(100):
        X_.append(Xt)
        if ((Xt==Xt_1) and (Xt_1==Xt_2) and (i%2==0)) or ((Xt==Xt_2)and (Xt_1==Xt_3)and (i%2==0)):
            break
        if typ == "out":
            State = (1-State)*(theta1 < Xt) + State*(theta2 > (1-Xt))
        if typ == "red":
            State = (1-State)*(theta1 < Xt) + State*(theta2 > ((1-Xt)/Xt))
        Xt_3 = Xt_2
        Xt_2 = Xt_1
        Xt_1 = Xt
        Xt = double_Integral(theta1, theta2, State*P)
    return X_

In [17]:
#Analytical Mean field
Simulator = ABM_MFana
name = "ResultsAna" #name to store results
Results = ResultsAna #point to propper results dict
save = True
run(Simulator,Results,name,save)


  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

0 out


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [08:43<00:00,  5.25s/it]


0 red


  3%|██▌                                                                             | 1/31 [14:27<7:13:52, 867.75s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████▍  | 30/31 [5:01:13<08:20, 500.84s/it]

10 out


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [03:27<00:00,  3.07s/it]


10 red


100%|███████████████████████████████████████████████████████████████████████████████| 31/31 [5:09:10<00:00, 493.64s/it]


In [13]:
#Analytical Mean field
Simulator = ABM_MFana
name = "ResultsAna" #name to store results
Results = ResultsAna #point to propper results dict
save = True
run(Simulator,Results,name,save)

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

-0.1 out


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [12:18<00:00, 11.51s/it]


-0.1 red


 14%|███████████▍                                                                    | 1/7 [21:59<2:11:54, 1319.16s/it]

...

 86%|█████████████████████████████████████████████████████████████████████▍           | 6/7 [1:22:41<11:40, 700.04s/it]

-0.7 out


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [04:16<00:00,  3.52s/it]


-0.7 red


100%|█████████████████████████████████████████████████████████████████████████████████| 7/7 [1:30:57<00:00, 638.69s/it]


In [57]:
def ABM_MFsim(par,typ):
    abm = ABM_MF(50000, par, phi2 = typ)
    abm.step(50)
    return abm.X

In [5]:
#Simulation Mean field
Simulator = ABM_MFsim
Results = ResultsSim 
name = "ResultsSim" 
save = False
run(Simulator,Results,name,save)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

0 out
0 red
0.05 out
0.05 red
0.1 out
0.1 red
0.15 out
0.15 red
0.2 out
0.2 red



  0%|                                                                                           | 0/50 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

 64%|████████████████████████████████████████████████████▍                             | 32/50 [54:17<18:41, 62.28s/it]C:\Users\snied\OneDrive\Dokumente\ETH\Agent-Based Modelling of Social Systems\ABM-Project-FS21\Classes.py:132: RuntimeWarning: divide by zero encountered in long_scalars
  return np.array(self.N*[(self.N-sum(self.s))/ np.sum(self.s)])

 50%|███████████████████████████████████████                                       | 5/10 [1:15:34<1:15:34, 906.81s/it]

0.25 out



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [30:52<00:00,  5.60s/it]
                                                                                                                       

...


 90%|█████████████████████████████████████████████████████████████████████▎       | 9/10 [8:19:30<1:25:13, 5113.01s/it]

0.45 out



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [31:29<00:00,  6.32s/it]
                                                                                                                       

0.45 red



100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [10:07:41<00:00, 5526.37s/it]


In [25]:
def ABM_Karate(par,typ):
    X_ = []
    for i in range(20):
        net = nx.karate_club_graph()
        abm = ABM(net, par, phi2 = typ)
        abm.step(50)
        X_.append(abm.X[-2:])
    return np.mean(X_, axis = 0)

In [27]:
#Karate
Simulator = ABM_Karate
Results = ResultsKarate 
name = "ResultsKarate" 
save = True
run(Simulator,Results,name,save)


  0%|                                                                                           | 0/31 [00:00<?, ?it/s]

0 out


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [05:35<00:00,  5.14s/it]


0 red


  3%|██▌                                                                             | 1/31 [15:53<7:56:52, 953.75s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████▍  | 30/31 [7:25:07<12:24, 744.59s/it]

10 out


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [05:35<00:00,  3.67s/it]


10 red


100%|███████████████████████████████████████████████████████████████████████████████| 31/31 [7:38:41<00:00, 765.37s/it]
